# FacePhi Challenge 2022

## Data preparation

### Downloading and extraction

In [ ]:
import os
import tqdm
import argparse
from urllib.request import urlretrieve
import tarfile
import zipfile
import os


midv500_links = [
    "ftp://smartengines.com/midv-500/dataset/01_alb_id.zip",
    "ftp://smartengines.com/midv-500/dataset/05_aze_passport.zip",
    "ftp://smartengines.com/midv-500/dataset/21_esp_id_old.zip",
    "ftp://smartengines.com/midv-500/dataset/22_est_id.zip",
    "ftp://smartengines.com/midv-500/dataset/24_fin_id.zip",
    "ftp://smartengines.com/midv-500/dataset/25_grc_passport.zip",
    "ftp://smartengines.com/midv-500/dataset/32_lva_passport.zip",
    "ftp://smartengines.com/midv-500/dataset/39_rus_internalpassport.zip",
    "ftp://smartengines.com/midv-500/dataset/41_srb_passport.zip",
    "ftp://smartengines.com/midv-500/dataset/42_svk_id.zip",
]

midv2019_links = [
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/01_alb_id.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/05_aze_passport.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/21_esp_id_old.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/22_est_id.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/24_fin_id.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/25_grc_passport.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/32_lva_passport.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/39_rus_internalpassport.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/41_srb_passport.zip",
    "ftp://smartengines.com/midv-500/extra/midv-2019/dataset/42_svk_id.zip",
]

midv2020_links = ["ftp://smartengines.com//midv-2020/dataset/photo.tar"]

def extract(path):
    out_path, extension = os.path.splitext(path)

    if extension == ".tar":
        with tarfile.open(path, "r:") as tar:
            tar.extractall(out_path, )
    elif extension == ".zip":
        with zipfile.ZipFile(path) as zf:
            zf.extractall(out_path)
    else:
        raise NotImplementedError()

class tqdm_upto(tqdm.tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download(url: str, save_dir: str):
    # Creates save_dir if it does not exist
    os.makedirs(save_dir, exist_ok=True)

    # Downloads the file
    with tqdm_upto(unit="B", unit_scale=True, miniters=1) as t: 
        urlretrieve(
            url,
            filename=os.path.join(save_dir, url.split("/")[-1]),
            reporthook=t.update_to,
            data=None,
        )

def download_and_extract(links_set, download_dir: str = './data'):
    out_path = os.path.join(download_dir)
    for i, link in enumerate(links_set):
        # download zip file
        link = link.replace("\\", "/")
        filename = os.path.basename(link)
        print()
        print(f"Downloading {i+1}/{len(links_set)}:", filename)
        download(link, out_path)

        # unzip zip file
        print("Unzipping:", filename)
        zip_path = os.path.join(out_path, filename)
        extract(zip_path)

        # remove zip file
        os.remove(zip_path)

download_and_extract(midv500_links, download_dir='data/midv500')
# download_and_extract(midv2019_links, download_dir='data/midv2019')
# download_and_extract(midv2020_links, download_dir='data/midv2020')

In [ ]:
from typing import List, Tuple
import glob
import tqdm
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
plt.rcParams["figure.figsize"] = (20, 16)


classes = [
    "alb_id",
    "aze_passport",
    "esp_id",
    "est_id",
    "fin_id",
    "grc_passport",
    "lva_passport",
    "rus_internalpassport",
    "srb_passport",
    "svk_id",
]

def get_class(*, img_path: str, dataset: str):
    if dataset in ['midv500', 'midv2019']:
        dirname = img_path.split('/')[-4]
        return '_'.join(dirname.split('_')[1:3])
    else:
        dirname = img_path.split('/')[-2]
        return '_'.join(dirname.split('_')[:2])

def get_location(*, loc_path: str):
    return json.load(open(loc_path, 'r'))['quad']

def get_location_path(img_path: str, loc_dirname: str = 'ground_truth'):
    loc_path = img_path.replace('images', loc_dirname)
    loc_path = os.path.splitext(loc_path)[0]+'.json'
    return loc_path

def get_metadata(image_paths: List[str], dataset: str, gt_dirname: str = 'ground_truth'):
    location_paths = [get_location_path(img_path=path, loc_dirname=gt_dirname) for path in image_paths]
    locations = np.stack(
        [get_location(loc_path=path) for path in location_paths],
        axis=0
    )
    class_labels = np.array([get_class(img_path=path, dataset=dataset) for path in image_paths])

    return locations, class_labels

def get_midv500_data(path='data/midv500'):
    image_paths = glob.glob(os.path.join(path, '*', '*', 'images', '*', '*'))
    locations, class_labels = get_metadata(image_paths, dataset='midv500')

    return image_paths, locations, class_labels

def get_midv2019_data(path='data/midv2019'):
    image_paths = glob.glob(os.path.join(path, '*', 'images', '*', '*'))
    locations, class_labels = get_metadata(image_paths, dataset='midv2019')

    return image_paths, locations, class_labels

def get_midv2020_data(path='data/midv2020'):
    gt_paths = glob.glob(os.path.join(path, 'photo', 'annotations', '*.json'))

    class_labels = []
    locations = []
    image_paths = []

    for gt_path in gt_paths:
        json_data = json.load(open(gt_path, 'r'))
        # class_name = json_data['_via_settings']['project']['name']
        class_name = os.path.splitext(os.path.basename(gt_path))[0]
        basedir = os.path.join(path, 'photo', 'images', class_name)

        for k, v in json_data['_via_img_metadata'].items():
            image_paths.append(os.path.join(basedir, v['filename']))
            for reg in v['regions']:
                if reg['shape_attributes']['name'] == 'polygon':
                    x = reg['shape_attributes']['all_points_x']
                    y = reg['shape_attributes']['all_points_y']
                    loc = np.stack([x, y], axis=1)
            locations.append(loc)
            class_labels.append(class_name)

    return image_paths, locations, class_labels

def show_image(image_path: str, location: np.ndarray = None, label: str = None):
    plt.imshow(load_img(image_path))

    if location is not None:
        x, y = location[:, 0], location[:, 1]
        plt.plot(np.append(x, x[0]), np.append(y, y[0]), color=(1, 0, 0), linewidth=2.0)
    if label is not None:
        print(label)
    plt.show()




### Preprocessing

In [ ]:
def load_img(path: str, size: Tuple[int, int] = None):
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    if size is not None:
        img = cv2.resize(img, size)
    return img

def preprocess(
    image_paths: List[str],
    locations: np.ndarray,
    labels: List[str],
    image_size: Tuple[int, int],
    class_names: str = None
):
    if class_names is None:
        unique_labels, label_ids = np.unique(labels, return_inverse=True)
    else:
        unique_labels = class_names
        label_ids = np.array([class_names.index(l) for l in labels])

    images = np.zeros((len(image_paths), image_size[0], image_size[1], 3), dtype=np.uint8)
    for i, path in enumerate(tqdm.tqdm(image_paths)):
        images[i] = load_img(path, size=image_size)

    # Normalize in range (-1, 1)
    images = images.astype(np.float32) / 127.5 - 1.0
    return images, label_ids, unique_labels


input_size = (224, 224)
image_paths, locations, labels = get_midv500_data()
train_images, train_label_ids, unique_labels = preprocess(image_paths, locations, labels, input_size)

### Visualization

In [ ]:
plt.rcParams["figure.figsize"] = (20,8)

idx = np.random.randint(len(train_images))
img = load_img(image_paths[idx], size=(224, 224))

plt.imshow(img)
plt.show()

## Experimentation

### Model definition

In [ ]:
import tensorflow as tf

def get_model(input_shape: Tuple[int, int, int], depth: int = 4, n_classes: int = 10):
    inp = x = tf.keras.layers.Input(input_shape)

    for d in range(depth):
        x = tf.keras.layers.Conv2D(2**(5+d), 3, padding='same', activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(n_classes)(x)

    return tf.keras.models.Model(inputs=inp, outputs=x)

model = get_model(input_shape=input_size+(3,), depth=4, n_classes=len(unique_labels))

model.summary()

### Training

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])


model.fit(train_images, train_label_ids, batch_size=64, epochs=10, shuffle=True)

## Evaluation

In [ ]:
download_and_extract(midv2020_links, download_dir='data/midv2020')

test_image_paths, test_locations, test_labels = get_midv2020_data()

test_images, test_label_ids, _ = preprocess(test_image_paths, test_locations, test_labels, input_size)

In [ ]:
preds = model.predict(test_images, batch_size=32, verbose=1)
preds = np.argmax(preds, axis=-1)

### Submission

In [ ]:
from sklearn.metrics import f1_score

print("MacroF1:", f1_score(y_true=test_label_ids, y_pred=preds, labels=np.arange(10), average='macro'))
print("MicroF1:", f1_score(y_true=test_label_ids, y_pred=preds, labels=np.arange(10), average='micro'))